In [11]:
from ipynb.fs.full.quick_certify import nearNeighborsSet, mostFreqLabel

In [12]:
import itertools

def generateSubsetsR1(data, k, input_x, n, min_rmv=0):
    """
    Generates all subsets of the input_set.
    
    Returns:
        list: A list of sets representing all subsets of the input_set.
    """
    
    input_set = nearNeighborsSet(data = data, k = k, input_x = input_x, n = n)
    
    all_subsets = []
    taille = len(input_set)
    for k in range(min_rmv, taille + 1):
        # Generate all combinations of size k from the input_set
        
        subsets_of_size_k = itertools.combinations(input_set, k)
        all_subsets.extend(map(list, subsets_of_size_k))
        
    return all_subsets


# data = [
#     ([1.2, 3.5], 'Star'),
#     ([2.5, 4.8], 'Star'),
#     ([3.7, 2.1], 'Star'),
#     ([5.1, 6.2], 'Triangle'),
#     ([6.9, 1.8], 'Triangle'),
#     ([7.5, 5.5], 'Triangle'),
#     ([9.2, 8.1], 'Star'),
#     ([8.5, 7.4], 'Triangle'),
#     ([0.5, 1.9], 'Star')
# ]
# n = 2
# k = 2
# input_x = [5, 4.5]
# resulting_subsets = generateSubsetsR1(data, k, input_x, n=1)
# for subset in resulting_subsets:
#     print(subset)


In [26]:
import numpy as np
import itertools
import copy  # Assuming you need to create a deep copy

def generateSubsetsR2(data, k, input_x, n, len_r2):
    """
    Generates all subsets of the input_set.

    Returns:
        list: A list of sets representing all subsets of the input_set.
    """

    nearDataSet = nearNeighborsSet(data=data, k=k, input_x=input_x, n=n)
    
    # input_set = [x for x in data if not np.any(x == element for element in nearDataSet)]
    # input_set = [x for x in data if not np.any(set(x) == set(element) for element in nearDataSet)]
    input_set = [x for x in data if not any(np.array_equal(x, element) for element in nearDataSet)]


    all_subsets = []

    for k in range(1, n - len_r2):
        # Generate all combinations of size k from the input_set using itertools.combinations
        subsets_of_size_k = itertools.combinations(input_set, k)
        all_subsets.extend(map(list, subsets_of_size_k))

    return all_subsets

from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

# Créer une liste de paires (x, y) à partir de X et y
xy = list(zip(X, y))

print(generateSubsetsR2(xy, 10, [5.1, 3.5, 1.4, 0.2], 2, len_r2=2))


[]


In [ ]:
def GenPromisingSubset(data, n, x, y):
    Kset = [1, 2, 3]
    reduced_set = []

    for k in Kset:
        start = 0
        end = n+1
        
        while start < end:
            mid = (start + end) // 2 # On prend la partie entiere du milieu
            # print(k, "/", x, "/",  y, "/", mid)
            # print(mostFreqLabel(data=data, k=k, input_x=x, y='0', n=mid))
            if y != mostFreqLabel(data=data, k=k, input_x=x, y='0', n=mid):
                end = mid
            else:
                start = mid + 1
        
        min_rmv = start
        print('start = ', start, 'end = ', end, 'mid = ', mid, 'n = ', n)
        print(min_rmv <= n)
        if (True):
            print('first step')
            
            for subsetR1 in generateSubsetsR1(data, k, x, n=n, min_rmv=min_rmv):
                print(generateSubsetsR2(data, k, x, n, len_r2=len(subsetR1)))
                for subsetR2 in generateSubsetsR2(data, k, x, n, len_r2=len(subsetR1)):
                    R = subsetR1 + subsetR2
                    print(R)
                    # print(R)
                    reduced_set = [w for w in data if w not in R]

    return reduced_set

from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

# Créer une liste de paires (x, y) à partir de X et y
xy = list(zip(X, y))
# print((xy))

# Afficher les 5 premières paires
print(GenPromisingSubset(data=xy, n=1, x=[5.1, 3.5, 1.4, 0.2], y=0))


In [25]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)

# Créer une liste de paires (x, y) à partir de X et y
xy = list(zip(X, y))
# print((xy))

# Afficher les 5 premières paires
print(GenPromisingSubset(data=xy, n=2, input_x=[5.1, 3.5, 1.4, 0.2], y=0))


[]
